In [43]:
# Packages / libraries
import os
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from math import sqrt
import json
import datetime as dt

In [44]:
# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

# Increases the size of sns plots
sns.set(rc={'figure.figsize':(5,5)})

# view all the dataframe
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# remove warnings
import warnings
warnings.filterwarnings("ignore")

In [45]:
# Loading the data

with open(r"accounton_data.json",'r') as f:
    raw_data = json.loads(f.read())

In [46]:
# Flatten data
data = pd.json_normalize(raw_data)

In [47]:
# convert the CreationDate column into DateTime type Delet the companies the created after 2019 because it had many NaN values
data['creation_date'] = pd.to_datetime(data['creation_date']).dt.to_period('Y')
data.head()

,company_name,nace_code,vat_number,zipcode,city,creation_date,legal_form,company_category,province,ebit.2020,ebit.2019,ebit.2018,ebit.2017,ebit.2016,ebit.2015,ebitda.2020,ebitda.2019,ebitda.2018,ebitda.2017,ebitda.2016,ebitda.2015,profit_and_loss_after_taxes.2020,profit_and_loss_after_taxes.2019,profit_and_loss_after_taxes.2018,profit_and_loss_after_taxes.2017,profit_and_loss_after_taxes.2016,profit_and_loss_after_taxes.2015,total_assets.2020,total_assets.2019,total_assets.2018,total_assets.2017,total_assets.2016,total_assets.2015,total_liabilities.2020,total_liabilities.2019,total_liabilities.2018,total_liabilities.2017,total_liabilities.2016,total_liabilities.2015,operating_profit_and_loss.2020,operating_profit_and_loss.2019,operating_profit_and_loss.2018,operating_profit_and_loss.2017,operating_profit_and_loss.2016,operating_profit_and_loss.2015,financial_profit_and_loss.2020,financial_profit_and_loss.2019,financial_profit_and_loss.2018,financial_profit_and_loss.2017,financial_profit_and_loss.2016,financial_profit_and_loss.2015,staff_count.2020,staff_count.2019,staff_count.2018,staff_count.2017,staff_count.2016,staff_count.2015,revenue.2020,revenue.2019,revenue.2018,revenue.2017,revenue.2016,revenue.2015,net_added_value.2020,net_added_value.2019,net_added_value.2018,net_added_value.2017,net_added_value.2016,net_added_value.2015,staff_costs.2020,staff_costs.2019,staff_costs.2018,staff_costs.2016,staff_costs.2017,staff_costs.2015
0,FREQUENSEA,47594,BE0765414330,8400,OOSTENDE,2021,Private company,None,West-Flanders,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,THE CONTENT MAKERS,82300,BE0685384578,9831,DEURLE,2017,Private company with limited liability,Small,East-Flanders,-5340.0,-1190.0,5361.0,NaN,NaN,NaN,-5220.0,-1190.0,6779.0,NaN,NaN,NaN,-5518.0,-1314.0,2705.0,NaN,NaN,NaN,4419.0,13004.0,12438.0,NaN,NaN,NaN,4419.0,13004.0,12438.0,NaN,NaN,NaN,-5340.0,-1190.0,5361.0,NaN,NaN,NaN,-178.0,-124.0,-162.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4643.0,-710.0,7800.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
2,NEW REUBENS SPORT,47785,BE0506833512,8340,SIJSELE,2014,Private company with limited liability,Small,West-Flanders,-1186.0,-11597.0,-14977.0,25787.0,12584.0,28042.0,1649.0,2460.0,-838.0,39432.0,25070.0,39473.0,-8670.0,-19038.0,-10320.0,16729.0,6369.0,14478.0,203025.0,186668.0,210797.0,269665.0,257450.0,279496.0,203025.0,186668.0,210797.0,269665.0,257450.0,279496.0,-1186.0,-11597.0,-14977.0,25787.0,12584.0,28042.0,-7300.0,-7129.0,-3884.0,-2895.0,-2691.0,-2448.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,9813.0,30898.0,33493.0,74107.0,61687.0,73463.0,7140.0,27664.0,33445.0,34954.0,32223.0,31226.0
3,DIEVERTECH,33120,BE0538788775,8490,ZERKEGEM,2013,Private company,Small,West-Flanders,62157.0,39586.0,4685.0,36303.0,51434.0,44802.0,82713.0,55349.0,65618.0,42285.0,60443.0,55171.0,44921.0,24155.0,359.0,25351.0,36195.0,31691.0,575080.0,554703.0,497592.0,194825.0,174172.0,124455.0,575080.0,554703.0,497592.0,194825.0,174172.0,124455.0,62157.0,39586.0,4685.0,36303.0,51434.0,44802.0,-9012.0,-10118.0,-4326.0,-1749.0,-1141.0,-953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84915.0,57471.0,66483.0,43259.0,63538.0,55781.0,NaN,NaN,NaN,0.0,NaN,0.0
4,TRAGURIUM,56101,BE0736478933,9100,SINT-NIKLAAS,2019,Private company,Small,East-Flanders,-2247.0,NaN,NaN,NaN,NaN,NaN,-2247.0,NaN,NaN,NaN,NaN,NaN,-10933.0,NaN,NaN,NaN,NaN,NaN,7983.0,NaN,NaN,NaN,NaN,NaN,7983.0,NaN,NaN,NaN,NaN,NaN,-2247.0,NaN,NaN,NaN,NaN,NaN,-48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30386.0,NaN,NaN,NaN,NaN,NaN,-1769.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# Save the rows tha have values .. deleting the rows that have all NAN values
df1 = data[data['ebit.2019'].notna()]

In [49]:
# Split the Nace_code to git the first 2 digits
df1['Nace_code'] = df1['nace_code'].str[0:2]

In [50]:
# making the reshape 
# create a dataframe to save and arrange the data 
reshape_df = pd.DataFrame()
# creat the final complete data frame
full_df = pd.DataFrame()
# determine the years and the features
years = ['2015', '2016','2017', '2018','2019']
Features = ['ebit', 'ebitda' , 'profit_and_loss_after_taxes' , 'total_assets' , 'total_liabilities' ,
    'operating_profit_and_loss' , 'financial_profit_and_loss' ,
    'staff_count' , 'net_added_value' , 'staff_costs']
# Loop to access the data from the original dataframe and assigh it in the new dataframe
for y in years:
                
                # reshape_df['vat_number'] = df1['vat_number']
                reshape_df['vat_number']= df1['vat_number']
                reshape_df['company_category'] = df1['company_category']
                reshape_df['province'] = df1['province']
                reshape_df['nace_code'] = df1['nace_code']
                reshape_df['Nace_code'] = df1['Nace_code']
                reshape_df['Year'] = y
                
                for f in Features:
                    reshape_df[f"{f}"] = df1[f"{f}.{y}"]
                reshape_df[f"current_revenue"] = df1[f"revenue.{y}"]
                reshape_df[f"next_year_revenue"] = df1[f"revenue.{str(int(y)+1)}"]
                full_df = pd.concat([full_df,reshape_df], axis=0)

In [67]:
full_df_after_drop = full_df.dropna(subset=['current_revenue'])
full_df_after_drop.shape

(31456, 18)

In [68]:
full_df_after_drop['current_revenue'].isnull().sum()

0

In [69]:
full_df_after_drop = full_df_after_drop.dropna(subset=['next_year_revenue'])
full_df_after_drop.shape

(29373, 18)

In [70]:
full_df_after_drop['next_year_revenue'].isnull().sum()

0

In [55]:
full_df_after_drop.sort_values(['vat_number','Year'])
#full_df_after_drop.head()

,vat_number,company_category,province,nace_code,Nace_code,Year,ebit,ebitda,profit_and_loss_after_taxes,total_assets,total_liabilities,operating_profit_and_loss,financial_profit_and_loss,staff_count,net_added_value,staff_costs,current_revenue,next_year_revenue
24710,BE0400000680,Small,Vlaams Brabant,41201,41,2015,23246.0,28427.0,23246.0,349023.0,349023.0,23246.0,NaN,2.0,110034.0,81607.0,322111.0,299959.0
24710,BE0400000680,Small,Vlaams Brabant,41201,41,2016,30902.0,35302.0,13390.0,396054.0,396054.0,30902.0,NaN,2.0,117744.0,82442.0,299959.0,324316.0
24710,BE0400000680,Small,Vlaams Brabant,41201,41,2017,31820.0,37227.0,24843.0,399514.0,399514.0,31820.0,NaN,2.0,116131.0,78904.0,324316.0,442699.0
24710,BE0400000680,Small,Vlaams Brabant,41201,41,2018,94826.0,102458.0,81124.0,331984.0,331984.0,94826.0,NaN,2.0,194245.0,91787.0,442699.0,182986.0
24710,BE0400000680,Small,Vlaams Brabant,41201,41,2019,33279.0,40911.0,8310.0,336241.0,336241.0,33279.0,NaN,1.0,73935.0,33024.0,182986.0,66548.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28471,BE0899980846,Small,Limburg,64200,64,2017,19459.0,38077.0,8410.0,592993.0,592993.0,19459.0,-11064.0,NaN,40547.0,0.0,50751.0,52922.0
28471,BE0899980846,Small,Limburg,64200,64,2018,23541.0,35963.0,10024.0,595020.0,595020.0,23541.0,-11473.0,NaN,38381.0,0.0,52922.0,82351.0
28471,BE0899980846,Small,Limburg,64200,64,2019,352.0,61844.0,90.0,842028.0,842028.0,352.0,-50.0,NaN,64285.0,0.0,82351.0,84943.0
35421,BE0899999256,Medium sized,East-Flanders,71111,71,2018,362283.0,420946.0,237387.0,1977229.0,1977229.0,362283.0,-12227.0,5.0,648680.0,226124.0,NaN,3827315.0


In [71]:
company_cat = ['Large', 'Medium sized','Small','Very large']
province =  ['Antwerp','East-Flanders','Limburg','Vlaams Brabant','West-Flanders']
nace_code_list = list(full_df_after_drop['Nace_code'])

years = ['2015', '2016','2017', '2018','2019']
Features = ['ebit', 'total_liabilities' , 'net_added_value' , 'staff_costs']

In [72]:
# fill the NAN values in each feature based on the median values of the same category in same provine and activites

for each_category in company_cat:
    for each_province in province:
        median_df =full_df_after_drop.groupby(['company_category', 'province', 'Nace_code']).median()
        i = median_df.loc[each_category, each_province]
        nace_list = i.index
        nace_list = list(nace_list)
        for nc in nace_list:
            for feature in Features:
                median = median_df.loc[(each_category, each_province, nc),f'{feature}']
                full_df_after_drop[f'{feature}'].fillna(median, inplace=True)

In [73]:
full_df_after_drop.isnull().sum()

vat_number                         0
company_category                   0
province                           0
nace_code                          0
Nace_code                          0
Year                               0
ebit                               0
ebitda                             3
profit_and_loss_after_taxes       16
total_assets                       7
total_liabilities                  0
operating_profit_and_loss          6
financial_profit_and_loss        614
staff_count                    15957
net_added_value                    0
staff_costs                        0
current_revenue                    0
next_year_revenue                  0
dtype: int64

In [ ]:
#df1.loc[(df1['staff_count.2015'].isna()) & (df1['staff_costs.2015']==0.0) ], df1['staff_costs.2015']